### 1) Parametry sesji

In [1]:
"""
Inicjalizacja parametrów runu i opis celu notatnika.

RUN_ID      – unikalny identyfikator bieżącej sesji
SEED        – ziarno losowości dla powtarzalności
PRECISION   – 'fp16' | 'fp32' | 'bf16' dla CLIP
TARGET_ENV  – 'local' | 'gcp' (wpływa na ścieżki I–O)
"""

from datetime import datetime, timezone

rec = {}  # inicjalizacja rekordu logu

ts = datetime.now(timezone.utc).isoformat(timespec="seconds")
rec["ts"] = ts

RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S_A1")
SEED = 42
PRECISION = "fp16"
TARGET_ENV = "local"  # na GCP ustawisz na 'gcp'

print({"run_id": RUN_ID, "precision": PRECISION, "target_env": TARGET_ENV})

{'run_id': '20251126_103155_A1', 'precision': 'fp16', 'target_env': 'local'}


### 2) Importy i wersje

In [2]:
"""
Importy, wersje bibliotek i minimalna telemetria środowiska.
"""

import os, sys, json, time, random, platform, math
from pathlib import Path
import numpy as np

# opcjonalni zależni – łagodna degradacja
try:
    import torch
except Exception as e:
    torch = None

try:
    import open_clip
except Exception:
    open_clip = None

try:
    from ultralytics import YOLO
except Exception:
    YOLO = None

try:
    from paddleocr import PaddleOCR
except Exception:
    PaddleOCR = None

try:
    from PIL import Image
except Exception as e:
    Image = None

try:
    import yaml
except Exception:
    yaml = None

try:
    from dotenv import load_dotenv
except Exception:
    load_dotenv = None

VERS = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "torch": getattr(torch, "__version__", None),
    "open_clip": getattr(open_clip, "__version__", None) if open_clip else None,
    "ultralytics": getattr(YOLO, "__module__", None) if YOLO else None,
    "paddleocr": getattr(PaddleOCR, "__module__", None) if PaddleOCR else None,
    "pillow": getattr(Image, "__version__", None) if Image else None,
    "pyyaml": getattr(yaml, "__version__", None) if yaml else None,
}

VERS

{'python': '3.13.7',
 'platform': 'macOS-26.1-arm64-arm-64bit-Mach-O',
 'torch': '2.9.0',
 'open_clip': '3.2.0',
 'ultralytics': 'ultralytics.models.yolo.model',
 'paddleocr': 'paddleocr._pipelines.ocr',
 'pillow': '12.0.0',
 'pyyaml': '6.0.2'}

### 3) Detekcja urządzenia i dtype

In [1]:
"""
Wybór urządzenia obliczeń i typu danych dla CLIP.

Zasady:
- Mac: mps>>cpu
- GCP: cuda>>cpu
- PRECISION steruje dtype (fp16 domyślnie)
"""

def get_device():
    if torch is None:
        return "cpu"
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps"
    if torch.cuda.is_available():
        return "cuda"
    return "cpu"

def get_dtype(precision: str):
    if torch is None:
        return None
    p = precision.lower()
    if p == "fp16":
        return torch.float16
    if p == "bf16":
        return torch.bfloat16
    return torch.float32

DEVICE = get_device()
DTYPE = get_dtype(PRECISION)
{"device": DEVICE, "dtype": str(DTYPE)}

NameError: name 'torch' is not defined

### 4) Seed i deterministyka

In [5]:
"""
Ustawienie seed i trybu deterministycznego tam, gdzie to rozsądne.
"""

random.seed(SEED)
np.random.seed(SEED)
if torch is not None:
    torch.manual_seed(SEED)
    if DEVICE == "cuda":
        torch.cuda.manual_seed_all(SEED)
    try:
        torch.use_deterministic_algorithms(False)  # True może spowalniać; wyłączone na bootstrapie
    except Exception:
        pass
{"seed": SEED}

{'seed': 42}

### 5) Ścieżki projektu i katalogi

In [6]:
"""
Definicja ścieżek bazowych i tworzenie katalogów zgodnie z konwencją projektu.
"""

ROOT = Path.cwd().resolve()
DIRS = {
    "configs": ROOT/"configs",
    "data_raw": ROOT/"data/raw",
    "data_staging": ROOT/"data/staging",
    "data_processed": ROOT/"data/processed",
    "models_clip": ROOT/"models/clip",
    "models_yolo": ROOT/"models/yolo",
    "out_previews": ROOT/"outputs/previews",
    "out_clip_epoch": ROOT/"outputs/clip_epoch",
    "out_yolo": ROOT/"outputs/yolo",
    "out_ocr": ROOT/"outputs/ocr",
    "exports": ROOT/"exports/dlibra_csv",
    "logs": ROOT/"logs",
    "logs_audit": ROOT/"logs/audit",
}

for p in DIRS.values():
    p.mkdir(parents=True, exist_ok=True)

{ k:str(v) for k,v in DIRS.items() }

{'configs': '/Users/olga/MetaLogic/configs',
 'data_raw': '/Users/olga/MetaLogic/data/raw',
 'data_staging': '/Users/olga/MetaLogic/data/staging',
 'data_processed': '/Users/olga/MetaLogic/data/processed',
 'models_clip': '/Users/olga/MetaLogic/models/clip',
 'models_yolo': '/Users/olga/MetaLogic/models/yolo',
 'out_previews': '/Users/olga/MetaLogic/outputs/previews',
 'out_clip_epoch': '/Users/olga/MetaLogic/outputs/clip_epoch',
 'out_yolo': '/Users/olga/MetaLogic/outputs/yolo',
 'out_ocr': '/Users/olga/MetaLogic/outputs/ocr',
 'exports': '/Users/olga/MetaLogic/exports/dlibra_csv',
 'logs': '/Users/olga/MetaLogic/logs',
 'logs_audit': '/Users/olga/MetaLogic/logs/audit'}

### 6) Loader .env i YAML + szablony konfigów

In [7]:
"""
Ładowanie .env (jeśli jest) oraz minimalnych plików YAML.
Tworzy szablony, gdy brak plików.
"""

# .env
if load_dotenv:
    load_dotenv()

# nadpisanie TARGET_ENV z .env (opcjonalnie)
TARGET_ENV = os.getenv("TARGET_ENV", TARGET_ENV)

def write_if_missing(path: Path, content: str):
    if not path.exists():
        path.write_text(content, encoding="utf-8")

clip_epoch_yaml = """\
model_name: "openclip:ViT-B-32"
precision: "fp16"
batch_size: 64
device: "auto"
prompts:
  template: "Zdjęcie z {label}."
  labels: ["lata 1940.", "lata 1950.", "lata 1960.", "lata 1970.", "lata 1980.", "po 1990"]
thresholds:
  accept_top1: 0.62
paths:
  cache_dir: "outputs/clip_epoch/cache"
seed: 42
"""

ui_yaml = """\
defaults:
  input_dir: "data/staging"
  n_preview: 24
  enable_yolo: false
  enable_ocr: true
  epoch_mode: "zero-shot"
  target_env: "%s"
""" % TARGET_ENV

write_if_missing(DIRS["configs"]/ "clip_epoch.yaml", clip_epoch_yaml)
write_if_missing(DIRS["configs"]/ "ui.yaml", ui_yaml)

{"TARGET_ENV": TARGET_ENV}

{'TARGET_ENV': 'local'}

### 7) Runlog helper

In [8]:
"""
Helper do zapisu kroków do logs/runlog.jsonl.

log_step(step, params, status, duration_s)
"""

RUNLOG_PATH = DIRS["logs"]/ "runlog.jsonl"

def log_step(step: str, params: dict, status: str, duration_s: float):
    rec = {
        "ts": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "run_id": RUN_ID,
        "step": step,
        "params": params,
        "status": status,
        "duration_s": round(float(duration_s), 3),
        "env": {
            "device": DEVICE,
            "precision": PRECISION,
            "dtype": str(DTYPE),
            "target_env": TARGET_ENV,
            "versions": VERS,
        },
    }
    with RUNLOG_PATH.open("a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

log_step("prepare_env", {"seed": SEED}, "ok", 0.0)
RUNLOG_PATH.as_posix()

/var/folders/k2/5zl19j_12ps2q88dx76v08_c0000gp/T/ipykernel_43873/2515235977.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat(timespec="seconds") + "Z",


'/Users/olga/MetaLogic/logs/runlog.jsonl'

### 8) Smoke-test I/O obrazów

In [9]:
"""
Test I/O: znajdź pierwszy obraz w data/raw lub data/staging, zrób miniaturę 512px,
zapisz do outputs/previews/<RUN_ID>_iocheck.jpg
"""

t0 = time.time()
preview_file = DIRS["out_previews"]/f"{RUN_ID}_iocheck.jpg"

found = None
for base in [DIRS["data_staging"], DIRS["data_raw"]]:
    for ext in ("*.jpg","*.jpeg","*.png","*.tif","*.tiff","*.bmp"):
        files = list(base.rglob(ext))
        if files:
            found = files[0]
            break
    if found:
        break

status = "skip"
if Image is None:
    status = "no_pillow"
elif found:
    try:
        im = Image.open(found)
        im = im.convert("RGB")
        im.thumbnail((512,512))
        im.save(preview_file, format="JPEG", quality=90)
        status = "ok"
    except Exception as e:
        status = f"error:{e}"
else:
    status = "no_images"

log_step("smoketest_io", {"input": str(found) if found else None, "preview": str(preview_file)}, status, time.time()-t0)
{"status": status, "preview": preview_file.as_posix()}

/var/folders/k2/5zl19j_12ps2q88dx76v08_c0000gp/T/ipykernel_43873/2515235977.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat(timespec="seconds") + "Z",


{'status': 'ok',
 'preview': '/Users/olga/MetaLogic/outputs/previews/20251112_124518_A1_iocheck.jpg'}

### 9) Smoke-test CLIP + FP16 zgodność

In [10]:
"""
Test CLIP: załaduj ViT-B/32 (open_clip), policz embedding obrazu i tekstu,
wypisz dtype i cos-sim. Wymuś spójność PRECISION.
"""

t0 = time.time()
status = "skip"
details = {}

if (open_clip is not None) and (torch is not None) and (Image is not None):
    try:
        model_name = "ViT-B-32"
        pretrained = "openai"
        model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=DEVICE)
        tokenizer = open_clip.get_tokenizer(model_name)

        # dtype i urządzenie
        if PRECISION.lower() == "fp16" and DTYPE == torch.float16 and DEVICE in ("cuda","mps"):
            model = model.to(dtype=torch.float16)
        elif PRECISION.lower() == "bf16" and DTYPE == torch.bfloat16 and DEVICE == "cuda":
            model = model.to(dtype=torch.bfloat16)
        else:
            model = model.to(dtype=torch.float32)

        # wybór obrazu do testu: weź ten sam co w smoketest_io albo syntetyczny
        test_img = None
        if 'found' in globals() and found:
            test_img = Image.open(found).convert("RGB")
        else:
            # syntetyczny 256x256
            from PIL import ImageDraw
            test_img = Image.new("RGB",(256,256),(128,128,128))
            d = ImageDraw.Draw(test_img)
            d.rectangle([64,64,192,192], outline=(255,255,255), width=4)

        img_tensor = preprocess(test_img).unsqueeze(0)
        txt = ["zdjęcie ulicy", "zdjęcie wnętrza"]
        tok = tokenizer(txt)

        # przerzut na device + dtype
        img_tensor = img_tensor.to(DEVICE)
        tok = tok.to(DEVICE)

        if model.visual.conv1.weight.dtype != img_tensor.dtype:
            # dopasuj dtype wejścia do modelu
            img_tensor = img_tensor.to(model.visual.conv1.weight.dtype)

        with torch.no_grad():
            img_feat = model.encode_image(img_tensor)
            txt_feat = model.encode_text(tok)
            # normalizacja
            img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
            txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)
            sim = (img_feat @ txt_feat.T).softmax(dim=-1).squeeze(0).tolist()

        details = {
            "model":"open_clip:ViT-B-32",
            "device": DEVICE,
            "model_dtype": str(next(model.parameters()).dtype),
            "input_dtype": str(img_tensor.dtype),
            "similarity": sim,
        }
        status = "ok"
    except Exception as e:
        status = f"error:{e}"
else:
    status = "deps_missing"

log_step("smoketest_clip", details, status, time.time()-t0)
{"status": status, **details}

/Users/olga/MetaLogic/.venv/lib/python3.13/site-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(
/var/folders/k2/5zl19j_12ps2q88dx76v08_c0000gp/T/ipykernel_43873/2515235977.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat(timespec="seconds") + "Z",


{'status': 'ok',
 'model': 'open_clip:ViT-B-32',
 'device': 'mps',
 'model_dtype': 'torch.float16',
 'input_dtype': 'torch.float16',
 'similarity': [0.4990234375, 0.5009765625]}

### 10) Smoke-test YOLO (opcjonalny)

In [11]:
"""
Test YOLO: jeśli 'ultralytics' dostępne, uruchom detekcję na jednym obrazie.
Nie zapisuj wyników – tylko liczba detekcji.
"""

ENABLE_YOLO_SMOKETEST = False  # ustaw True, gdy chcesz sprawdzić YOLO
if ENABLE_YOLO_SMOKETEST:
    t0 = time.time()
    status = "skip"
    det_count = None
    if YOLO is not None and 'found' in globals() and found:
        try:
            model = YOLO("yolov8n.pt")
            res = model.predict(source=str(found), imgsz=640, conf=0.25, verbose=False,
                                device=0 if DEVICE=="cuda" else None)
            det_count = int(sum(len(r.boxes) for r in res))
            status = "ok"
        except Exception as e:
            status = f"error:{e}"
    else:
        status = "deps_missing_or_no_image"
    log_step("smoketest_yolo",
             {"input": str(found) if 'found' in globals() else None, "det_count": det_count},
             status, time.time()-t0)
    {"status": status, "det_count": det_count}
else:
    print("YOLO smoketest pominięty (ENABLE_YOLO_SMOKETEST=False)")

YOLO smoketest pominięty (ENABLE_YOLO_SMOKETEST=False)


### 11) Smoke-test OCR (opcjonalny)

### 12) Podsumowanie sesji

In [12]:
"""
Podsumowanie 00_bootstrap: urządzenie, precyzja, pliki wyjściowe, wskazówki dalszych kroków.
"""

summary = {
    "run_id": RUN_ID,
    "device": DEVICE,
    "precision": PRECISION,
    "dtype": str(DTYPE),
    "target_env": TARGET_ENV,
    "created_dirs": [str(p) for p in DIRS.values()],
    "runlog": RUNLOG_PATH.as_posix(),
    "preview_hint": f"outputs/previews/{RUN_ID}_iocheck.jpg"
}
print(json.dumps(summary, indent=2, ensure_ascii=False))

print("\nNastępne kroki:")
print("1) 01_ingest.ipynb – przygotowanie danych.")
print("2) 02_clip_epoch_zero_shot.ipynb – zero-shot dekady.")
print("3) 06_merge_export.ipynb – scalanie i eksport CSV.")

{
  "run_id": "20251112_124518_A1",
  "device": "mps",
  "precision": "fp16",
  "dtype": "torch.float16",
  "target_env": "local",
  "created_dirs": [
    "/Users/olga/MetaLogic/configs",
    "/Users/olga/MetaLogic/data/raw",
    "/Users/olga/MetaLogic/data/staging",
    "/Users/olga/MetaLogic/data/processed",
    "/Users/olga/MetaLogic/models/clip",
    "/Users/olga/MetaLogic/models/yolo",
    "/Users/olga/MetaLogic/outputs/previews",
    "/Users/olga/MetaLogic/outputs/clip_epoch",
    "/Users/olga/MetaLogic/outputs/yolo",
    "/Users/olga/MetaLogic/outputs/ocr",
    "/Users/olga/MetaLogic/exports/dlibra_csv",
    "/Users/olga/MetaLogic/logs",
    "/Users/olga/MetaLogic/logs/audit"
  ],
  "runlog": "/Users/olga/MetaLogic/logs/runlog.jsonl",
  "preview_hint": "outputs/previews/20251112_124518_A1_iocheck.jpg"
}

Następne kroki:
1) 01_ingest.ipynb – przygotowanie danych.
2) 02_clip_epoch_zero_shot.ipynb – zero-shot dekady.
3) 06_merge_export.ipynb – scalanie i eksport CSV.
